In [1]:
import requests
import defs
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle("instruments.pkl")

In [4]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD','CAD']

In [5]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [6]:
code, res = fetch_candles("EUR_USD",9,"H1")

In [7]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 7541,
   'time': '2024-09-20T16:00:00.000000000Z',
   'bid': {'o': '1.11584', 'h': '1.11751', 'l': '1.11582', 'c': '1.11731'},
   'mid': {'o': '1.11591', 'h': '1.11759', 'l': '1.11590', 'c': '1.11738'},
   'ask': {'o': '1.11598', 'h': '1.11767', 'l': '1.11597', 'c': '1.11745'}},
  {'complete': True,
   'volume': 4886,
   'time': '2024-09-20T17:00:00.000000000Z',
   'bid': {'o': '1.11729', 'h': '1.11736', 'l': '1.11575', 'c': '1.11585'},
   'mid': {'o': '1.11736', 'h': '1.11744', 'l': '1.11582', 'c': '1.11592'},
   'ask': {'o': '1.11744', 'h': '1.11751', 'l': '1.11590', 'c': '1.11600'}},
  {'complete': True,
   'volume': 5106,
   'time': '2024-09-20T18:00:00.000000000Z',
   'bid': {'o': '1.11587', 'h': '1.11654', 'l': '1.11565', 'c': '1.11643'},
   'mid': {'o': '1.11594', 'h': '1.11662', 'l': '1.11572', 'c': '1.11651'},
   'ask': {'o': '1.11602', 'h': '1.11669', 'l': '1.11580', 'c': '1.11659'}},

In [8]:
code

200

In [9]:
def get_candles_df(json_response):
    
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)


In [10]:
get_candles_df(res)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-09-20T16:00:00.000000000Z,7541,1.11591,1.11759,1.11590,1.11738,1.11584,1.11751,1.11582,1.11731,1.11598,1.11767,1.11597,1.11745
1,2024-09-20T17:00:00.000000000Z,4886,1.11736,1.11744,1.11582,1.11592,1.11729,1.11736,1.11575,1.11585,1.11744,1.11751,1.11590,1.11600
2,2024-09-20T18:00:00.000000000Z,5106,1.11594,1.11662,1.11572,1.11651,1.11587,1.11654,1.11565,1.11643,1.11602,1.11669,1.11580,1.11659
3,2024-09-20T19:00:00.000000000Z,4100,1.11650,1.11660,1.11617,1.11620,1.11643,1.11653,1.11609,1.11613,1.11658,1.11668,1.11625,1.11628
4,2024-09-20T20:00:00.000000000Z,1921,1.11621,1.11643,1.11561,1.11636,1.11613,1.11631,1.11553,1.11625,1.11629,1.11661,1.11569,1.11646
5,2024-09-22T21:00:00.000000000Z,572,1.11628,1.11662,1.11605,1.11624,1.11578,1.11638,1.11578,1.11602,1.11678,1.11686,1.11621,1.11645
6,2024-09-22T22:00:00.000000000Z,1045,1.11619,1.11648,1.11607,1.11622,1.11593,1.11640,1.11590,1.11614,1.11645,1.11657,1.11618,1.11630
7,2024-09-22T23:00:00.000000000Z,1452,1.11622,1.11639,1.11590,1.11591,1.11614,1.11631,1.11582,1.11583,1.11629,1.11647,1.11598,1.11599


In [11]:
def save_file(candles_df,pair,granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [12]:
def create_data(pair,granularity):
    code ,json_data = fetch_candles(pair,4000,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,pair, granularity)

In [13]:
for p1 in our_curr:
    for p2 in our_curr:
        pair =  f"{p1}_{p2}"
        if pair in ins_df.name.unique():
          create_data(pair,"H1")

EUR_USD loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
USD_JPY loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
USD_CHF loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
USD_CAD loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
GBP_USD loaded 3999 candles from 2024-02-01T08:00:00.000000000Z to 2024-09-22T23:00:00.000000000Z
GBP_JPY loaded 3999 

In [14]:
ins_df.shape[0]

68

In [15]:
ins_df

,name,type,displayName,pipLocation,marginRate
0,USD_CNH,CURRENCY,USD/CNH,-4,0.05
1,NZD_CAD,CURRENCY,NZD/CAD,-4,0.03
2,EUR_CHF,CURRENCY,EUR/CHF,-4,0.04
3,SGD_CHF,CURRENCY,SGD/CHF,-4,0.05
4,EUR_CZK,CURRENCY,EUR/CZK,-4,0.05
...,...,...,...,...,...
63,USD_SEK,CURRENCY,USD/SEK,-4,0.03
64,GBP_SGD,CURRENCY,GBP/SGD,-4,0.05
65,EUR_PLN,CURRENCY,EUR/PLN,-4,0.05
66,HKD_JPY,CURRENCY,HKD/JPY,-4,0.1


In [16]:
ins_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [17]:
ins_df.columns


Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [18]:
ins_df.name

0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object

In [19]:
ins_df.name.unique

<bound method Series.unique of 0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object>